In [ ]:
from config import *

In [ ]:
from pyedflib import EdfReader

class EdfReaderWrapper(EdfReader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def __enter__(self):
        return self
    def __exit__(self, *args):
        super().close()

In [ ]:
import os
tuh_sz_path = os.path.join(dataset_path, "edf")
plot_args_path = timeline_info_path; os.makedirs(plot_args_path, exist_ok=True)

import os
import json, csv
import glob
from datetime import datetime, timedelta
import pyedflib
import pandas as pd

with os.scandir(tuh_sz_path) as entries:
    for entry in entries:
        if entry.is_dir():
            # subdir_path = entry.path
            with os.scandir(entry.path) as pats:
                for pat in pats:
                    if pat.is_dir():
                        result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
                        record_fn_lst =  glob.glob(os.path.join(pat.path, "**", "*.edf"), recursive=True)
                        
                        for edf_path in record_fn_lst:
                            with EdfReaderWrapper(edf_path) as pedf: 
                                start_dt = pedf.getStartdatetime()
                                end_dt = start_dt + timedelta(seconds=(edf_len := pedf.getFileDuration()))
                                fs = pedf.getSampleFrequency(0)
                                # assert all((FS := pedf.getSampleFrequencies()) == fs) # TODO 支持过滤非脑电数据通道
                                result_obj["record_lst"].append({
                                    "file": os.path.basename(edf_path), 
                                    # "span": [start_dt.strftime(dt_fmt), end_dt.strftime(dt_fmt)], 
                                    "span": [start_dt, end_dt], 
                                    "info": f"{os.path.basename(edf_path)} records {start_dt} ~ {end_dt} \r\n of shape {pedf.signals_in_file, pedf.getNSamples()[0]}"
                                })
                            
                            with open(edf_path[:-3]+"csv_bi", "rt") as fcsv:
                                rows = (row for row in fcsv if not row.startswith('#') and row.strip() != '')
                                reader = csv.DictReader(rows)
                                for row in reader:
                                    if row["label"] == "bckg":
                                        continue
                                    # TODO Assertion Needed Here
                                    # assert row["label"] == "seiz"
                                    # assert row["channel"] == "TERM"
                                    # assert 0 <= float(row["start_time"]) < float(row["stop_time"]) <= edf_len
                                    result_obj["seizure_lst"].append({
                                        "span": [start_dt+timedelta(seconds=float(row["start_time"])), start_dt+timedelta(seconds=float(row["stop_time"]))], 
                                        "info": f'Onset {start_dt+timedelta(seconds=float(row["start_time"]))}, last {float(row["stop_time"]) - float(row["start_time"])}s'
                                    })
                            # TODO Add more verification
                        result_obj["record_lst"].sort(key=lambda obj:obj["span"])
                        result_obj["seizure_lst"].sort(key=lambda obj:obj["span"])
                        for k, rec_info in enumerate(result_obj["record_lst"]):
                            if rec_info["file"] in ignore_lst:
                                result_obj["unused_rec_idx_lst"].append(k)
                            rec_info["span"] = [rec_info["span"][0].strftime(dt_fmt), rec_info["span"][1].strftime(dt_fmt)]
                        
                        for seiz_info in result_obj["seizure_lst"]:
                            seiz_info["span"] = [seiz_info["span"][0].strftime(dt_fmt), seiz_info["span"][1].strftime(dt_fmt)]
                            
                        with open(os.path.join(plot_args_path, f'{pat.name}_sz{len(result_obj["seizure_lst"])}_edf{len(result_obj["record_lst"])}.json'), "wt") as fout:
                            json.dump(result_obj, fout, indent=2)

